# Математика в алгоритмах кластеризации

## k-means

**Формулировка задачи:**

Даны:
* $X$ — это пространство объектов. 
* $X^N = {x_1,...,x_N}$ - обучающая выборка.
* $\rho:X\times X \rightarrow [0,\infty)$ - функуия расстояния на $X$

Найти:
* *Y* - множество кластеров
* Алгоритм $a:X\rightarrow Y$ т.ч.
    * каждый кластер состоит из "близких между собой" объектов по расстоянию $\rho$
    * объекты из разных кластеров "отличаются существенно" по расстоянию $\rho$

**алгоритма Ллойда**

* Зададим параметр $K=|Y|$
* Обзначим $\mu _k$ - центры кластеров $k \in Y$
* Задаём начальные приближения $\mu_k$ для центров (центроидов) всех кластеров $k$. Это может быть сделано случайно или же как-то по-другому и зависит от вариации алгоритма Ллойда
* Повторояем:
    1. Отнесем каждый объект $x_i$ из обучающей выборки к кластеру, расстояние до центроида которого минимально:
        $a(x_i) = argmin_{k\in Y} \rho(x_i - \mu _k)$. Где $i \in\{1,...,N\}$, $a: X \rightarrow Y$ - значение, которым мы положим наш алгоритм на данный момент
    2. Пересчитываем центроиды $\mu_k =  \frac{\sum_{i=1}^N [a(x_i)=k] x_i}{\sum_{i=1}^N [a(x_i)=k]}$. Эта формула, если разобраться, просто означает среднее арифметическое координат по элементам из данного кластера.
* Алгоритм останавливаем, когда координаты значения a на двух подряд повторая полностью совпадают по всем $x_i$. Т.е. никакие объекты не изменили кластер
* И еще бы весь алгоритм повторить несколько раз с новыми начальными приближениями, а потом выбрать "лучший" из всех результатов

**Реализации в sklearn**

Классический вариант

In [1]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
#n_clusters — количество кластеров;
#n_init — количество итераций алгоритма k-means;
#random_state — параметр для воспроизводимости результатов от запуска к запуску.

MINI-BATCH K-MEANS

Данная вариация k-means используется, когда данных очень много. Из-за их объёма вычисление центров по всей выборке занимает много времени.

Для решения этой проблемы k-means на каждом шаге работает с небольшой подвыборкой данных. В общем случае упрощённый алгоритм должен сходиться к тому же результату, что и на полной выборке. Однако исследования показывают, что качество кластеров может ухудшаться по сравнению с классическим k-means.

In [2]:
from sklearn.cluster import MiniBatchKMeans
kmeans = MiniBatchKMeans(n_clusters=2,random_state=42,batch_size=6)
#Отличие лишь в том, что можно дополнительно настроить объём подвыборки batch_size

K-MEANS++

Данную вариацию k-means используют, если признаков очень много.

Результат и время работы алгоритма зависят от изначального выбора центроидов. Чтобы минимизировать затраты, будем действовать следующим образом:

1. Первый центроид выбираем случайным образом.
2. Для каждой точки вычисляем квадрат расстояния до ближайшего центроида из тех, что уже поставлены.
3. Далее из этих точек выбираем следующий центроид так, чтобы вероятность выбора точки была пропорциональна вычисленному для неё квадрату расстояния.
4. Когда все точки выбраны, реализуем k-means.

По умолчанию при запуске k-means в sklearn используется именно алгоритм k-means++. Выбор алгоритма задаётся через параметр init:

init='random' — для классической версии k-means;
init='k-means++' — для вариации k-means++.

**Выбор целевого количества классов K**

Введем функцию **инерции** разбиения с количеством кластеров K = сумме квадратов всех расстояний от точек до центров кластеров к которым они принадлежат:

$J(K) = \sum_{k=1}^K \sum_{a(x)=k} \rho (x, \mu _k) $

Тут обычно $\rho = ||x - \mu _k ||^2$

In [3]:
#inertia = kmeans.inertia_

Одной стороны, чем меньше инерция, тем лучше, но минимум очевидно достгается, когда у нас число кластеров = числу объектов.
Поэтому обычно ищут **по методу Локтя** такое К, что отношение модуля приращения инерции при переходе от $K \rightarrow K+1$ к модулю приращения инерции при переходе от $K-1 \rightarrow K$ минимально (это и будет точка локтя): $\frac{\left|J\left(K\right)-J\left(K+1\right)\right|}{\left|J\left(K-1\right)-J\left(K\right)\right|} \rightarrow \min _{K}$

Если точка локтя неочевидна, то можно исполльзовать **коэффицент силуэта**: $s_i=\frac{(b_i-a_i)}{max(a_i,b_i)}$, где

$a_i$ - среднее расстояние от $x_i$ до точек своего кластера, $b_i$ - среднее расстояние от $x_i$ до точек ближайшего несвоего кластера

Для получения итогового значения рассчитывается среднее значение силуэта для всего датасета.

In [4]:
from sklearn.metrics import silhouette_score
def get_silhouette(cluster_num, X):
    k_means =  KMeans(n_clusters=cluster_num, random_state=42)
    k_means.fit(X)
# подсчитаем метрику силуэта, передав данные и то, к каким кластерам относятся объекты
    silhouette = silhouette_score(X, k_means.labels_)
    return silhouette

silhouette = []
#for clust_num in range(2, 10):
#    silhouette.append(get_silhouette(clust_num, X))

Это значение меняется от -1 до 1. Чем ближе он к 1, тем больше различия между кластерами в дата сете. Т.е. кластеры максимально компактны внутри себя относительно своего разброса между собой. Нас интересует K, при котором силуэт максимально близок к 1

## EM-алгоритм

 EM (Expectation-maximization) — модель гауссовой смеси (Gaussian Mixture Model, GMM)

Предположим, что у нас есть набор некоторых объектов и мы знаем, что они получены из нескольких различных распределений Гаусса. Разумеется, знай мы, какие точки из какого распределения Гаусса получены, мы смогли бы использовать их для того, чтобы найти параметры каждого распределения. С другой стороны, если бы мы знали параметры этих распределений Гаусса, то могли бы предположить, к какому распределению относятся рассматриваемые точки.

Таким образом, нам необходимы параметры распределения, чтобы идентифицировать принадлежность точек, и необходимо знать принадлежность точек, чтобы вычислить параметры распределения. Такой парадокс немного напоминает извечный риторический вопрос о курице и яйце. Но если проверить, что возникло раньше — курица или яйцо — мы не сможем, то разобраться с нашей дилеммой нам позволит ЕМ-алгоритм.

Для реализации алгоритма в sklearn мы используем GaussianMixture. Для запуска алгоритма GaussianMixture необходимо задать следующие основные параметры:

n_components — количество кластеров;
random_state — так как в алгоритме есть случайность при инициализации, то для воспроизводимости результатов от запуска к запуску следует передать какое-то число.

## DBSCAN

Алгоритм DBSCAN инициализируется двумя параметрами:

* eps — расстояние, определяющее окрестности. Две точки считаются достаточно близкими, чтобы находиться в одном кластере, если расстояние между ними меньше или равно eps. В библиотеке sklearn оно по умолчанию равно 0.5.
* min_samples — минимальное количество точек в данных, которое может быть в кластере. В библиотеке sklearn оно по умолчанию равно 5.

Эти параметры очень сильно влияют на результат кластеризации, и следует перебирать разные варианты, т. к. их подходящие значения (особенно eps) может быть сложно подобрать. Не стоит опираться только на результаты алгоритма, обученного со значениями по умолчанию. 

Точка является **центральной**, если в её окрестностях имеется не менее min_samples точек (включая саму точку). Окрестность определяется как область внутри окружности радиуса eps, центр которой находится в центральной точке.

Точка является **граничной**, если она достижима из центральной точки (то есть находится в окрестности центральной точки) и количество точек в её окрестностях меньше min_samples.

Точка называется **шумовой**, если она не является центральной и не достижима ни из одной из центральных точек (то есть не находится в окрестностях центральных точек).

## Агломеративная иерархическая кластеризация

Алгоритм ОЧЕНЬ простой.
1. Выбираем два самых близких по расстоянию объекта. Объединяем их в новый "суммарный объект". Расстояние для суммарных объектов можно по-разному определять.
2. Переходим к итерации 1)) Только надо расстояния пересчитать с учетом того, что 2 объекта заменилиь на один суммарный.
3. Тем самым мы рисуем дерево от лисьев в корню (это значить агломеративная, можно в другую сторону пойти и будет "дивизионная")

Расстояние можно вычислять различными способами. В библиотеке sklearn для вычисления расстояний предлагается несколько вариаций — вот наиболее популярные из них:

1. Евклидово расстояние $d\left(a, b\right)=\sqrt{\sum_{i}^{N}\left(a_i-b_i\right)^{2}}$
2. Расстояние городских кварталов (манхэттенское расстояние): $d\left(a, b\right)=\sum_{i}^{N}\left|a_{i}-b_{i}\right|$ - по сути это мы едем из точки до точки не по прямой, а сначала по одной оси координат, а потом по другой
3. Косинусная мера близости. Это метрика, которая показывает не расстояние, а близость векторов (насколько маленький угол между ними). Чем больше её значение, тем ближе векторы друг к другу (максимально близки они в том случае, если совпадают). В числителе находится скалярное произведение, а в знаменателе — произведение длин векторов: $d(a, b)=\frac{\langle a, b\rangle}{\|a\| \cdot\|b\|}$

Теперь нас интересует следующий вопрос: как вычислить расстояния между двумя кластерами, если хотя бы в одном из них более одного объекта? То есть нам нужна метрика, как-то обобщающая попарные расстояния между точками кластеров. Таких метрик несколько.

1. Single linkage — метод одиночной связи (минимум попарных расстояний между точками из двух кластеров): $\min \{d(a, b): a \in A, b \in B\}$
2. Complete linkage — метод полной связи (максимум попарных расстояний между точками из двух кластеров): $\max \{d(a, b): a \in A, b \in B\}$
3. Average linkage — метод средней связи (среднее арифметическое попарных расстояний между точками из двух кластеров): $\frac{1}{\left|A \right| *\left|B \right|} \sum_{a \in A} \sum_{b \in B} d(a, b)$
4. Centroid linkage — центроидный метод (расстояние между центроидами двух кластеров):

Для реализации алгоритма в sklearn вам понадобится AgglomerativeClustering. Обычно ему передают следующие параметры:

* n_clusters — количество кластеров; по умолчанию — 2.
* linkage — метод определения расстояния между кластерами, которое мы рассматривали выше.
* affinity — метод определения расстояний между точками (например, евклидово или манхэттенское).